In [ ]:
%pip install keras_tuner

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [3]:
import os
import pandas as  pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
import keras
from tensorflow.keras import layers
import keras_tuner
import datetime

c:\Users\hugho\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
files = os.listdir('../final_stats/preprocessed_3/')
# no_sent_files = files[:7]
files[0]

'preprocessed_no_sent_last_1.csv'

### File Retrieval and Preprocessing

In [5]:
def split_target(df, target_col='away_winner_wts'):
    df = df.copy()
    target = df.pop(target_col)
    return df, target

def normalize_df(df):
    scaler = StandardScaler()
    data, target = split_target(df)
    data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    return data, target

def get_data(file):
    df = pd.read_csv(f'../final_stats/preprocessed_3/{file}', index_col=[0])
    data, target = normalize_df(df)
    return data, target

def shuffle_data(x,y, seed):
    x_shuffle = x.sample(frac=1, random_state=seed)
    y_shuffle = y.sample(frac=1, random_state=seed)
    return x_shuffle, y_shuffle


### Chart Code

In [6]:
def get_average_acc_history(acc_histories):
    num_epochs = len(acc_histories[0])
    return [np.mean([x[i] for x in acc_histories]) for i in range(num_epochs)]

def get_average_loss_history(loss_histories):
    num_epochs = len(loss_histories[0])
    return [np.mean([x[i] for x in loss_histories]) for i in range(num_epochs)]
        
def plot_ave_acc_and_loss_histories(acc_histories, loss_histories):
    average_acc_history = get_average_acc_history(acc_histories)
    average_loss_history = get_average_loss_history(loss_histories)

    plt.figure(figsize=(15,5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(average_acc_history) + 1), average_acc_history)
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")

    plt.subplot(1, 2, 2)

    plt.plot(range(1, len(average_loss_history) + 1), average_loss_history)
    plt.xlabel("Epochs")
    plt.ylabel("Validation Loss")
    plt.show()

def compare_ave_loss_acc_histories(acc_hist_1, acc_hist_2, loss_hist_1, loss_hist_2):
    min_epochs = min(len(acc_hist_1[0]), len(acc_hist_2[0]))
    
    ave_acc_hist_1 = get_average_acc_history(acc_hist_1)
    ave_loss_hist_1 = get_average_loss_history(loss_hist_1)
    
    ave_acc_hist_2 = get_average_acc_history(acc_hist_2)
    ave_loss_hist_2 = get_average_loss_history(loss_hist_2)


    blue_dots = 'bo'
    solid_blue_line = 'b'
    red_dots = 'ro'
    solid_red_line = 'r'

    epochs = range(1, min_epochs + 1)
    plt.figure(figsize=(15,5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, ave_acc_hist_1, solid_blue_line, label='Model 1')
    plt.plot(epochs, ave_acc_hist_2, solid_red_line, label='Model 2')
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)

    plt.plot(epochs, ave_loss_hist_1, solid_blue_line, label='Model 1')
    plt.plot(epochs, ave_loss_hist_2, solid_red_line, label='Model 2')
    plt.xlabel("Epochs")
    plt.ylabel("Validation Loss")
    plt.legend()
    
    plt.show()
    

### Helper Functions


In [7]:
def get_max_ave_acc_min_ave_loss(acc_histories, loss_histories):
    num_epochs = len(acc_histories[0])
    average_acc_history = [
        np.mean([x[i] for x in acc_histories]) for i in range(num_epochs)
    ]
    index_acc = np.argmax(average_acc_history)
    max_acc = round(average_acc_history[index_acc]*100,4)
    print(f'\nMax Average Accuracy: {max_acc}% \t Num Epochs: {index_acc + 1} ')

    average_loss_history = [
        np.mean([x[i] for x in loss_histories]) for i in range(num_epochs)
    ]
    index_loss = np.argmin(average_loss_history)
    min_loss = round(average_loss_history[index_loss], 4)
    print(f'Min Average Loss: {min_loss} \t Num Epochs: {index_loss + 1} ')
    return min_loss, max_acc

### Build Models

In [8]:
def build_custom_model(hps):
    model = keras.Sequential()
    for n in range(hps[0]):
        model.add(layers.Dense(hps[1], activation='tanh'))
        model.add(layers.Dropout(hps[2]))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=hps[3]),
        loss = 'binary_crossentropy',
        metrics='accuracy'
    )
    return model

In [102]:
def get_val_and_train(data, i, num_val_samples):
    val = data[i * num_val_samples: (i + 1) * num_val_samples]
    train = np.concatenate(
        [
            data[:i * num_val_samples],
            data[(i + 1) * num_val_samples:]
        ],
        axis=0
        )
    return val, train


def get_max_val_acc_epoch(val_acc):
    # max_val_acc = round(max(val_acc),5)*100
    return max(val_acc)


def run_k_fold_params(x, y, build_m, params, num_epochs=30, k=10, n_repeats=10):
    num_val_samples = len(x) // k
    max_scores = [] 
    
    all_histories = []
    acc_histories = []
    loss_histories = []
    curr_fold = 0
    total_folds = k*n_repeats
    # total fold s = 10*10 = 100
    # should get 100 max_accuracies = 100 max_scores
    all_scores = []
    for n in range(n_repeats):
        xs, ys = shuffle_data(x,y,n)
        for i in range(k):
            curr_fold = curr_fold + 1
            if curr_fold % 10 == 0:
                print(f"Fold {curr_fold}/{total_folds}")
 
            X_val, X_train = get_val_and_train(xs, i, num_val_samples)
            y_val, y_train = get_val_and_train(ys, i, num_val_samples)

            model = build_m(params)
            
            # history = model.fit(
            #         X_train, y_train, 
            #         validation_data=(X_val, y_val),
            #         epochs=num_epochs, batch_size=16, verbose=0
            #     )
            model.fit(X_train, y_train, epochs=num_epochs, verbose=0)
            val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
            print('VAL_ACC',val_acc)
            max_scores.append(val_acc)
            
            
            # all_histories.append(history.history)
            # acc_histories.append(history.history['val_accuracy'])
            # loss_histories.append(history.history['val_loss'])
            # max_scores.append(get_max_val_acc_epoch(history.history['val_accuracy']))

    # get_max_ave_acc_min_ave_loss(acc_histories, loss_histories)
    # plot_ave_acc_and_loss_histories(acc_histories, loss_histories)
    # return [all_histories, acc_histories, loss_histories]
    return max_scores

In [10]:
best_params_1 = [
    [6, 512, 0.3, 0.0185486],
    [6, 512, 0.2, 0.0185486],
    [5, 512, 0.3, 0.0185486],
    [6, 512, 0.2, 0.013],
    [5, 512, 0.2, 0.0185486]
]
best_params_2 = [
    [4, 256, 0.3, 0.00185486],
    [8, 256, 0.4, 0.0185486],
    [5, 1024, 0.3, 0.0185486],
    [2, 2048, 0.3, 0.013],
    [10, 128, 0.1, 0.0001]
]
best_params_3 = [
    [4, 256, 0.2, 0.00185486],
    [4, 256, 0.4, 0.0185486],
    [4, 512, 0.3, 0.0185486],
    [4, 256, 0.3, 0.013],
    [4, 256, 0.3, 0.0001]
]
best_params_4 = [
    [4, 256, 0.3, 0.03],
    [4, 256, 0.3, 0.1]
]

In [ ]:
no_sent_files = files[:7]
no_sent_files

In [103]:
def get_max_scores(histories):
    max_scores = []
    for h in histories:
        max_score = get_max_val_acc_epoch(h)
        max_scores.append(max_score)
    ms_mean = np.mean(max_scores)
    ms_std = np.std(max_scores)
    return max_scores, ms_mean, ms_std

In [104]:
def test_file(file_name, best_params):
    x,y = get_data(file_name)

    all_scores =[]
    max_score = {
        'scores_mean':0
    }

    for params in best_params:
        k_folds = 10
        n_repeats = 10
        epochs = 25
        # hist length should len k*n_repeats
        # hist = []
        scores = run_k_fold_params(x,y,build_custom_model, params, epochs, k_folds, n_repeats)
        # if len(hist[0] == k_folds*n_repeats):
        #     print('length of hist same as k_fold * n_repeats')
        # hist = run_k_fold_params(x,y,build_custom_model, params, 25, 10, 10)
        # print(k_fold_hist[1])
        # scores, scores_mean, scores_std = get_max_scores(k_fold_hist[1])

        # param_hists.append(k_fold_hist)
        # loss, acc = get_max_ave_acc_min_ave_loss(k_fold_hist[1], k_fold_hist[2])
        scores_dict = {
            'file': file_name,
            'scores_mean': np.mean(scores),
            'scores_std': np.std(scores),
            # 'acc': acc,
            # 'loss': loss,
            'layers': params[0],
            'units': params[1],
            'dropout_rate': params[2],
            'learn_rate': params[3],
            'scores': scores,
        }
        all_scores.append(scores_dict)
        if np.mean(scores) > max_score['scores_mean']:
            max_score = scores_dict
    df = pd.DataFrame.from_records(all_scores)
    df.to_csv(f'../results/nn1/{file_name}_nn_res.csv', index='file')
    # param results and hists should have length num params * k * n_repeats
    return all_scores, max_score


In [98]:
def get_all_files(files, best_params):
    all_results = []
    max_results = []
    for f in files:
        all_scores, max_score = test_file(f, best_params)
        max_results.append(max_score)
        all_results.append(all_scores)
    df = pd.DataFrame.from_records(max_results)
    df.to_csv('../results/nn1/nn_sent_results.csv', mode='a', header=True)
    return all_results, max_results
    


### Parameter Checking

In [39]:
all_results4 = get_all_files(no_sent_files, best_params_4)

Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 54.2% 	 Num Epochs: 3 
Min Average Loss: 0.99 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 51.6% 	 Num Epochs: 2 
Min Average Loss: 1.55 	 Num Epochs: 5 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 51.6% 	 Num Epochs: 2 
Min Average Loss: 0.88 	 Num Epochs: 5 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 53.2% 	 Num Epochs: 14 
Min Average Loss: 1.42 	 Num Epochs: 9 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.4% 	 Num Epochs: 19 
Min Average Loss: 1.04 	 Num Epochs: 5 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.6% 	 Num Epochs: 3 
Min Average Loss: 1.57 	 Num Epochs: 5 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 55.4% 	 Num Epochs: 3 
Min Average Loss: 0.97 	 Num Epochs: 4 
Fold 10/50


In [35]:
all_results2 = get_all_files(no_sent_files, best_params_3)

Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 57.2% 	 Num Epochs: 20 
Min Average Loss: 0.94 	 Num Epochs: 1 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 53.8% 	 Num Epochs: 17 
Min Average Loss: 1.01 	 Num Epochs: 4 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 55.2% 	 Num Epochs: 6 
Min Average Loss: 0.88 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 55.6% 	 Num Epochs: 14 
Min Average Loss: 1.14 	 Num Epochs: 4 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.8% 	 Num Epochs: 20 
Min Average Loss: 0.72 	 Num Epochs: 4 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.2% 	 Num Epochs: 16 
Min Average Loss: 0.95 	 Num Epochs: 1 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 49.6% 	 Num Epochs: 1 
Min Average Loss: 1.05 	 Num Epochs: 3 
Fold 10/

In [32]:
all_results2 = get_all_files(no_sent_files, best_params_2)

Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 57.4% 	 Num Epochs: 12 
Min Average Loss: 0.83 	 Num Epochs: 1 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 55.8% 	 Num Epochs: 7 
Min Average Loss: 0.77 	 Num Epochs: 7 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 53.4% 	 Num Epochs: 7 
Min Average Loss: 1.21 	 Num Epochs: 10 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.4% 	 Num Epochs: 12 
Min Average Loss: 2.64 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 54.8% 	 Num Epochs: 17 
Min Average Loss: 0.71 	 Num Epochs: 3 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 53.2% 	 Num Epochs: 10 
Min Average Loss: 0.91 	 Num Epochs: 1 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 54.2% 	 Num Epochs: 3 
Min Average Loss: 0.79 	 Num Epochs: 9 
Fold 10/

In [29]:
all_results = get_all_files(no_sent_files, best_params_1)

Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 53.0% 	 Num Epochs: 29 
Min Average Loss: 0.9 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 51.4% 	 Num Epochs: 14 
Min Average Loss: 0.94 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 55.2% 	 Num Epochs: 1 
Min Average Loss: 0.98 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.0% 	 Num Epochs: 1 
Min Average Loss: 0.91 	 Num Epochs: 6 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 55.4% 	 Num Epochs: 27 
Min Average Loss: 0.94 	 Num Epochs: 5 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 53.6% 	 Num Epochs: 4 
Min Average Loss: 0.95 	 Num Epochs: 10 
Fold 10/50
Fold 20/50
Fold 30/50
Fold 40/50
Fold 50/50

Max Average Accuracy: 52.4% 	 Num Epochs: 21 
Min Average Loss: 0.91 	 Num Epochs: 7 
Fold 10/5

### Final Test With Sentiment of Game 7 Wtd

In [22]:
files_test_7_wt = [f for f in files if 'last_7_wt' in f]
files_test_7_wt


['preprocessed_no_sent_last_7_wt.csv',
 'preprocessed_sent_24_last_7_wt.csv',
 'preprocessed_sent_96_last_7_wt.csv',
 'preprocessed_sent_cross_last_7_wt.csv']

#### Top 5 Hyperparameters

In [23]:
best_params_top5 = [
    [6, 512, 0.2, 0.0185486],
    [4, 256, 0.3, 0.00185486],
    [4, 256, 0.2, 0.00185486],
    [4, 256, 0.3, 0.0185486],
    [10, 128, 0.1, 0.0001]
]

#### Run for 10 folds and 10 repeats for each file and each param combination
Each file will have a total of 500 models trained
Take top accuracy from one model for comparison between each

In [24]:
files_test_7_wt[3]

'preprocessed_sent_cross_last_7_wt.csv'

In [106]:
res_all_fixed, res_max_fixed = get_all_files(files_test_7_wt, best_params_top5)

VAL_ACC 0.5
VAL_ACC 0.30000001192092896
VAL_ACC 0.6000000238418579
VAL_ACC 0.4000000059604645
VAL_ACC 0.699999988079071
VAL_ACC 0.699999988079071
VAL_ACC 0.5
VAL_ACC 0.4000000059604645
VAL_ACC 0.5
Fold 10/100
VAL_ACC 0.699999988079071
VAL_ACC 0.30000001192092896
VAL_ACC 0.6000000238418579
VAL_ACC 0.4000000059604645
VAL_ACC 0.699999988079071
VAL_ACC 0.6000000238418579
VAL_ACC 0.30000001192092896
VAL_ACC 0.5
VAL_ACC 0.6000000238418579
VAL_ACC 0.5
Fold 20/100
VAL_ACC 0.699999988079071
VAL_ACC 0.4000000059604645
VAL_ACC 0.5
VAL_ACC 0.699999988079071
VAL_ACC 0.5
VAL_ACC 0.6000000238418579
VAL_ACC 0.5
VAL_ACC 0.6000000238418579
VAL_ACC 0.4000000059604645
VAL_ACC 0.20000000298023224
Fold 30/100
VAL_ACC 0.5
VAL_ACC 0.5
VAL_ACC 0.4000000059604645
VAL_ACC 0.5
VAL_ACC 0.5
VAL_ACC 0.6000000238418579
VAL_ACC 0.800000011920929
VAL_ACC 0.699999988079071
VAL_ACC 0.699999988079071
VAL_ACC 0.30000001192092896
Fold 40/100
VAL_ACC 0.699999988079071
VAL_ACC 0.5
VAL_ACC 0.699999988079071
VAL_ACC 0.400000005

In [25]:
results_test, hists_test = get_all_files(files_test_7_wt[3:], best_params_top5)

Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 52.6% 	 Num Epochs: 1 
Min Average Loss: 0.91 	 Num Epochs: 14 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 56.2% 	 Num Epochs: 1 
Min Average Loss: 0.83 	 Num Epochs: 1 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 55.3% 	 Num Epochs: 20 
Min Average Loss: 0.91 	 Num Epochs: 1 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 53.4% 	 Num Epochs: 21 
Min Average Loss: 0.99 	 Num Epochs: 3 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 57.0% 	 Num Epochs: 14 
M

In [50]:
all_results_7wt = get_all_files(files_test_7_wt, best_params_top5)

Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 51.8% 	 Num Epochs: 29 
Min Average Loss: 0.9 	 Num Epochs: 8 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 56.4% 	 Num Epochs: 26 
Min Average Loss: 0.88 	 Num Epochs: 1 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 56.0% 	 Num Epochs: 13 
Min Average Loss: 0.89 	 Num Epochs: 1 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 54.8% 	 Num Epochs: 19 
Min Average Loss: 0.92 	 Num Epochs: 3 
Fold 10/100
Fold 20/100
Fold 30/100
Fold 40/100
Fold 50/100
Fold 60/100
Fold 70/100
Fold 80/100
Fold 90/100
Fold 100/100

Max Average Accuracy: 53.4% 	 Num Epochs: 16 
M

In [51]:
all_results_7wt

[[{'data': 'preprocessed_no_sent_last_7_wt.csv',
   'acc': 51.8,
   'loss': 0.9,
   'layers': 6,
   'units': 512,
   'dropout_rate': 0.2,
   'learn_rate': 0.0185486},
  {'data': 'preprocessed_no_sent_last_7_wt.csv',
   'acc': 56.4,
   'loss': 0.88,
   'layers': 4,
   'units': 256,
   'dropout_rate': 0.3,
   'learn_rate': 0.00185486},
  {'data': 'preprocessed_no_sent_last_7_wt.csv',
   'acc': 56.0,
   'loss': 0.89,
   'layers': 4,
   'units': 256,
   'dropout_rate': 0.2,
   'learn_rate': 0.00185486},
  {'data': 'preprocessed_no_sent_last_7_wt.csv',
   'acc': 54.8,
   'loss': 0.92,
   'layers': 4,
   'units': 256,
   'dropout_rate': 0.3,
   'learn_rate': 0.0185486},
  {'data': 'preprocessed_no_sent_last_7_wt.csv',
   'acc': 53.4,
   'loss': 0.71,
   'layers': 10,
   'units': 128,
   'dropout_rate': 0.1,
   'learn_rate': 0.0001}],
 [{'data': 'preprocessed_sent_24_last_7_wt.csv',
   'acc': 52.2,
   'loss': 0.89,
   'layers': 6,
   'units': 512,
   'dropout_rate': 0.2,
   'learn_rate': 0.01